# Fake news classification
![](https://static.scientificamerican.com/sciam/cache/file/DFD3D397-B132-4516-9D8D0C7B4F07763B_source.jpg?w=690&h=930&2A4ADDE6-C642-4536-8CBD4821540D9D7F)

In [ ]:
import pandas as pd

columns = [
    "text",
    "language",
    "thread_title",
    "spam_score",
    "replies_count",
    "participants_count",
    "likes",
    "comments",
    "shares",
    "type",
]
df = pd.read_csv("../input/fake-news-dataset/fake_news_dataset.csv", usecols=columns)

In [ ]:
df.head()

In [ ]:
df = df[df["language"] == "english"]
df = df.dropna()

In [ ]:
df = df.drop("language", axis=1)

In [ ]:
features = 0
feature_map = {}

# adds a feature to a dictionary of features
def add_feature(name):
    if name not in feature_map:
        global features
        feature_map[name] = features
        features += 1

add_feature("fake")
add_feature("real")

In [ ]:
# Wether article is fake or real
def article_type(row):
    if row["type"] == "fake":
        return feature_map["fake"]
    else:
        return feature_map["real"]

In [ ]:
df["type"] = df.apply(article_type, axis=1)

In [ ]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer_text = TfidfVectorizer()
vectorizer_title = TfidfVectorizer()


In [ ]:
vectorized_text = vectorizer_text.fit_transform(df_train.pop("text").values)

In [ ]:
vectorized_title = vectorizer_title.fit_transform(df_train.pop("thread_title").values)

In [ ]:
from scipy import sparse

spam_score_train = sparse.csr_matrix(df_train["spam_score"].values).transpose()
replies_count_train = sparse.csr_matrix(df_train["replies_count"].values).transpose()
participants_count_train = sparse.csr_matrix(
    df_train["participants_count"].values
).transpose()
likes_train = sparse.csr_matrix(df_train["likes"].values).transpose()
comments_train = sparse.csr_matrix(df_train["comments"].values).transpose()
shares_train = sparse.csr_matrix(df_train["shares"].values).transpose()

In [ ]:
from scipy.sparse import hstack

X_train = hstack(
    [
        vectorized_text,
        vectorized_title,
        spam_score_train,
        replies_count_train,
        participants_count_train,
        likes_train,
        comments_train,
        shares_train,
    ]
)
y_train = df_train.pop("type").values

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
vectorized_text_test = vectorizer_text.transform(df_test.pop("text").values)
vectorized_title_test = vectorizer_title.transform(df_test.pop("thread_title").values)

In [ ]:

spam_score_test = sparse.csr_matrix(df_test["spam_score"].values).transpose()
replies_count_test = sparse.csr_matrix(df_test["replies_count"].values).transpose()
participants_count_test = sparse.csr_matrix(
    df_test["participants_count"].values
).transpose()
likes_test = sparse.csr_matrix(df_test["likes"].values).transpose()
comments_test = sparse.csr_matrix(df_test["comments"].values).transpose()
shares_test = sparse.csr_matrix(df_test["shares"].values).transpose()

In [ ]:

X_test = hstack(
    [
        vectorized_text_test,
        vectorized_title_test,
        spam_score_test,
        replies_count_test,
        participants_count_test,
        likes_test,
        comments_test,
        shares_test,
    ]
)
y_test = df_test.pop("type").values

In [ ]:

clf.score(X_test, y_test)